# Day 1

## puzzle 1

In [3]:
with open("AOC_input_1.txt") as f:
    most_calories = 0
    calories = 0
    for line in f:
        if not line.startswith("\n"):
            calories += int(line.rstrip())
        else:
            if calories > most_calories:
                most_calories = calories
            calories = 0
most_calories

67027

## puzzle 2

In [4]:
with open("AOC_input_1.txt") as f:
    calories_list = []
    calories = 0
    for line in f:
        if not line.startswith("\n"):
            calories += int(line.rstrip())
        else:
            calories_list.append(calories)
            calories = 0
sum(sorted(calories_list, reverse=True)[:3])

197291

## refactor puzzles

In [5]:
def calorie_counter(input_file, num_elves):
    with open(input_file) as f:
        calories_list = []
        calories = 0
        for line in f:
            if not line.startswith("\n"):
                calories += int(line)
            else:
                calories_list.append(calories)
                calories = 0
    return sum(sorted(calories_list, reverse=True)[:num_elves])
    
calorie_counter("AOC_input_1.txt", 3)

197291